In [1]:
import requests
from alphacast import Alphacast
import pandas as pd
import time
from clave_api_alphacast import api_key

In [2]:
alphacast = Alphacast(api_key)

In [3]:
tc_serie_alpha = alphacast.datasets.dataset(5288).download_data("pandas")
tc_serie_alpha

,Date,country,ADR GGAL,BLUE,Dolar CCL,Dolar MEP,Dolar Mayorista,Dolar Oficial,GGAL,Merval,...,Brecha CCL - 30d_change,Brecha Blue - 30d_change,Dolar Oficial - 30d_change,Dolar Solidario,Dolar Solidario - 30d_running_av,Dolar Solidario - 30d_running_av_mom_change,Dolar Solidario - 30d_change,Dolar Solidario - usd_daily,Dolar Solidario - 7d_running_av,Dolar Solidario - 1d_change
0,1992-01-02,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-01-03,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-01-06,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-01-07,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-01-08,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8539,2023-12-07,Argentina,16.94,965.0,991.83,986.37,364.4083,400.500,1657.00,930497.09,...,0.186999,0.120222,0.088167,1021.28,848.763500,0.165957,0.397100,2.809187,971.004286,5.673341
8540,2023-12-08,Argentina,17.50,NaN,991.66,NaN,NaN,NaN,NaN,NaN,...,0.273484,0.120222,0.088167,NaN,858.439500,0.174336,0.397100,NaN,979.747857,NaN
8541,2023-12-11,Argentina,17.45,975.0,1000.55,993.53,366.0000,462.400,1756.35,949524.67,...,0.151399,-0.004484,0.256351,1021.28,887.467500,0.214046,0.397100,2.808029,1005.432143,NaN
8542,2023-12-12,Argentina,17.48,1045.0,1032.95,1011.46,366.5000,587.570,1780.85,1013725.72,...,0.207632,0.107613,0.596441,1021.28,897.143500,0.227282,0.397100,2.793283,1013.447143,0.000000


In [4]:
df = pd.read_excel('../output/ITCRM historico.xlsx',sheet_name="tc_equilibrio")
tc_read_df = pd.read_excel('../output/Tc a precios de hoy.xlsx')[['Período','dolar','inflacion','tc_real_hoy']]

In [5]:
tc_serie_alpha = tc_serie_alpha.set_index("Date")
tc_serie_alpha.index = pd.to_datetime(tc_serie_alpha.index, format = "%Y-%m-%d")
tc_serie_alpha = tc_serie_alpha[["BLUE", "Dolar CCL", "Dolar MEP", "Dolar Solidario"]]
df = df.set_index("Período")

In [6]:
df_brechas = df.merge(tc_serie_alpha, how = "outer", left_index = True, right_index = True)
df_brechas["blue-equilibrio"] = (df_brechas["BLUE"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["ccl-equilibrio"] = (df_brechas["Dolar CCL"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["mep-equilibrio"] = (df_brechas["Dolar MEP"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["solidario-equilibrio"] = (df_brechas["Dolar Solidario"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]

In [7]:
df

,tc_equilibrio,tc_oficial_mayorista,brecha
Período,,,
1997-11-24,1.924512,1.00,0.924512
1997-11-25,1.927250,1.00,0.927250
1997-11-26,1.932110,1.00,0.932110
1997-11-27,1.933234,1.00,0.933234
1997-11-28,1.932836,1.00,0.932836
...,...,...,...
2023-12-09,714.777015,385.00,0.856564
2023-12-10,717.214342,385.00,0.862894
2023-12-11,723.598821,365.95,0.977316


In [8]:
df_brechas

,tc_equilibrio,tc_oficial_mayorista,brecha,BLUE,Dolar CCL,Dolar MEP,Dolar Solidario,blue-equilibrio,ccl-equilibrio,mep-equilibrio,solidario-equilibrio
1992-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-09,714.777015,385.00,0.856564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-10,717.214342,385.00,0.862894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-11,723.598821,365.95,0.977316,975.0,1000.55,993.53,1021.28,0.347432,0.382741,0.373040,0.411390
2023-12-12,726.033973,366.45,0.981263,1045.0,1032.95,1011.46,1021.28,0.439327,0.422730,0.393130,0.406656


In [9]:
df_brechas.index[-1].strftime("%d/%m/%Y")

'13/12/2023'

In [10]:
import plotly.graph_objects as go

fig = go.Figure()
df_brechas = df_brechas["2011":]
ultima_fecha = df_brechas.index[-1].strftime("%d/%m/%Y")
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["solidario-equilibrio"], name = "Solidario"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["blue-equilibrio"], name = "Blue"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["ccl-equilibrio"], name = "CCL"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["mep-equilibrio"], name = "MEP"))
fig.update_yaxes(tickformat = ".2%")
fig.update_layout(template = None, 
                  title_text = f"Brecha entre el tc de equilibrio y las distintas cotizaciones<br><sup>Datos hasta {ultima_fecha}")


In [11]:
# alphacast.datasets.dataset(37268).download_data("pandas")
alphacast.datasets.dataset(37268).upload_data_from_df(df, deleteMissingFromDB = True, onConflictUpdateDB = True, uploadIndex=True)
time.sleep(10)
alphacast.datasets.dataset(37447).upload_data_from_df(tc_read_df, deleteMissingFromDB = True, onConflictUpdateDB = True, uploadIndex=True)

b'{"id": 3252218, "status": "Requested", "createdAt": "2023-12-14T14:32:44.791710", "datasetId": 37447}'

In [12]:
time.sleep(3)
alphacast.datasets.dataset(37268).processes()


b'[{"id": 3252217, "datasetId": 37268, "status": "Processed", "statusDescription": "143 values had conflict and were updated./n21 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-12-14T14:32:33", "processedAt": "2023-12-14T14:32:36"}, {"id": 3168266, "datasetId": 37268, "status": "Processed", "statusDescription": "564 values had conflict and were updated./n43 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-11-23T19:15:15", "processedAt": "2023-11-23T19:15:19"}, {"id": 2992963, "datasetId": 37268, "status": "Processed", "statusDescription": "0 values added to database./nDataset has not changed. Avoiding csv version upload.", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-10-12T18:12:49", "processedAt": "2023-10-12T18:28:52"}, {"id": 2992799, "datasetId": 37268, "status": "Processed", "statusDescription": "520 values had conflict and were updated./n23 values added

In [13]:
time.sleep(10)
alphacast.datasets.dataset(37447).processes()

b'[{"id": 3252218, "datasetId": 37447, "status": "Processed", "statusDescription": "526 values had conflict and were updated./nWARNING: More than 10%% of values changed. This may indicate a problem with the data./n1 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-12-14T14:32:44", "processedAt": "2023-12-14T14:32:47"}, {"id": 3168267, "datasetId": 37447, "status": "Processed", "statusDescription": "522 values had conflict and were updated./nWARNING: More than 10%% of values changed. This may indicate a problem with the data./n2 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-11-23T19:15:27", "processedAt": "2023-11-23T19:15:30"}, {"id": 2992964, "datasetId": 37447, "status": "Processed", "statusDescription": "0 values added to database./nDataset has not changed. Avoiding csv version upload.", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-10-12T18:13:01", "proce

In [14]:
# time.sleep(10)
# alphacast.datasets.dataset(37268).download_data("pandas")